In [1]:
import os
%pwd

'/Users/vikrantsaini/workspace/deep_learning/character_level_classification/research'

In [2]:
os.chdir("../")
%pwd


'/Users/vikrantsaini/workspace/deep_learning/character_level_classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from characterLevelClassification.constants import *
from characterLevelClassification.utils.common import read_yaml, create_directories

In [5]:
CONFIG_FILE_PATH

PosixPath('config/config.yaml')

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [7]:
import os
import urllib.request as request
import zipfile
from characterLevelClassification.logging import logger
from characterLevelClassification.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.validate_all_files_exist()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-08-11 15:45:22,004: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-08-11 15:45:22,007: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-11 15:45:22,009: INFO: common: created directory at: artifacts]
[2025-08-11 15:45:22,009: INFO: common: created directory at: artifacts/data_ingestion]
[2025-08-11 15:45:23,701: INFO: 1434958058: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 2882130
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "463fcd46d9f7c98c881d97047ae2e608c6887916554168e8ebda06c634b6939e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B96E:11597A:C42E8:1CE598:6899C2BA
Accept-Ranges: bytes
Date: Mon, 11 Aug 2025 10:15:23 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10235-MAA
X-Cache: MI